![](https://import.cdn.thinkific.com/220744/BExaQBPPQairRWFqxFbK_logo_mastermind_web_png)

¡Ahora es **vuestro turno!**

En este proyecto final os he adjuntado un Dataset con 27.075 juegos en **Steam**, la popular plataforma de juegos de PC. Esta lista está actualizada desde el inicio hasta Abril de 2019.

Ya que tenéis los conocimientos, os pediré que me contestéis a **10 preguntas**:
1. Cual es el TOP10 de juegos más jugados?
2. Cuál es el género de videojuegos más vendido?
3. Qué géneros tienen mejores reviews de media?
4. Cuánto ha ganado el desarrollador que más juegos ha vendido?
5. Cómo afecta a las reviews que un juego sea Early Access?
6. Cuál es el desarrollador (o desarrolladores) que más tipos de juego y cantidad ha desarrollado?
7. En qué mes, desde que tenemos datos, se han publicado más juegos?
8. Cuántas horas se juega de media a cada tipo de juego mensualmente?
9. Los juegos Free to Play tienen mejores o peores reviews?
10. Cómo ha evolucionado el número de achievements (trofeos) por juego?

Os dejamos via libre para contestar a estas preguntas como más os apetezca. Recordad que no hay nada de malo en consultar por internet si tenéis dudas. Las documentaciones oficiales o [Stack Overflow](https://stackoverflow.com/) son sitios geniales para ello!

Al acabar este proyecto, publicadlo en el foro donde está colgado para ver quién es el mejor!

Os dejo la guía de pasos a seguir.

Muchos ánimos!!




# Importando librerías

In [1]:
import urllib.request as Req
from zipfile import ZipFile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from thefuzz import process,fuzz
from unidecode import unidecode
import pdb

%matplotlib inline

# Consiguiendo los datos

### Conseguir link

In [2]:
url = 'https://neo-mastermind.s3.amazonaws.com/uploads/froala_editor/files/SteamDataset-220424-201858.zip'

### Descargar el archivo

In [3]:
Req.urlretrieve(url, 'steam.zip')

('steam.zip', <http.client.HTTPMessage at 0x267cf4b3ec0>)

### Descomprimir el archivo

In [4]:
steam_zip = ZipFile('steam.zip')

In [5]:
steam_zip.filelist

[<ZipInfo filename='steam.csv' compress_type=deflate external_attr=0x20 file_size=3425223 compress_size=979195>]

In [6]:
archivo = steam_zip.open('steam.csv')

### Construir DataFrame

In [7]:
df = pd.read_csv(archivo, index_col=0)

# Limpieza y manipulación de datos

## Quitar datos duplicados o irrelevantes

In [8]:
df.duplicated().any()

True

In [9]:
df.duplicated().sum()

28

In [10]:
df = df.drop_duplicates()

In [11]:
df.columns

Index(['name', 'release_date', 'developer', 'achievements', 'positive_ratings',
       'negative_ratings', 'avg_hours_per_user', 'price', 'sells', 'action',
       'adventure', 'rpg', 'simulation', 'strategy', 'racing', 'free_to_play',
       'early_access'],
      dtype='object')

In [12]:
df.drop('free_to_play', axis=1, inplace=True) # Redundante si tenemos la columna price, con la cual también generaba disparidades

## Arreglar errores estructurales

In [45]:
df.head()

,name,release_date,developer,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price €,sells,action,adventure,rpg,simulation,strategy,racing,early_access
0,Counter-Strike,2000-11-01,Valve,0,124534,3339,17612,7.19,15000000,True,False,False,False,False,False,False
1,Team Fortress Classic,1999-04-01,Valve,0,3318,633,277,3.99,7500000,True,False,False,False,False,False,False
2,Day of Defeat,2003-05-01,Valve,0,3416,398,187,3.99,7500000,True,False,False,False,False,False,False
3,Deathmatch Classic,2001-06-01,Valve,0,1273,267,258,3.99,7500000,True,False,False,False,False,False,False
4,Half-Life: Opposing Force,1999-11-01,Gearbox Software,0,5250,288,624,3.99,7500000,True,False,False,False,False,False,False


In [42]:
df.tail()

,name,release_date,developer,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price €,sells,action,adventure,rpg,simulation,strategy,racing,early_access
29318,Room of Pandora,2019-04-24,SHEN JIAWEI,7,3,0,0,2.09,10000,False,True,False,False,False,False,False
29319,Cyber Gun,2019-04-23,Semyon Maximov,0,8,1,0,1.69,10000,True,True,False,False,False,False,False
29320,Super Star Blast,2019-04-24,EntwicklerX,24,0,1,0,3.99,10000,True,False,False,False,False,False,False
29321,New Yankee 7: Deer Hunters,2019-04-17,Yustas Game Studio,0,2,0,0,5.19,10000,False,True,False,False,False,False,False
29322,Rune Lord,2019-04-24,Adept Studios GD,0,4,0,0,5.19,10000,False,True,False,False,False,False,False


In [43]:
df.sample(5)

,name,release_date,developer,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price €,sells,action,adventure,rpg,simulation,strategy,racing,early_access
7022,Escape From BioStation,2017-07-03,Tusky Games,11,15,2,0,0.79,10000,True,True,False,False,False,False,False
21142,Ostalgie: The Berlin Wall,2018-03-25,Kremlingames,55,284,18,571,4.49,75000,False,False,False,True,True,False,False
18868,Sneaky Bears,2017-10-17,WarDucks,24,13,0,0,6.19,10000,True,True,False,False,False,False,False
22433,Art of Murder - FBI Confidential,2018-07-13,City Interactive S A,0,19,1,0,3.99,10000,False,True,False,False,False,False,False
13395,The Spirit Underneath,2017-01-02,Acuze Interactives,0,28,60,216,0.79,350000,False,True,False,False,False,False,False


### Definir errores

Recuerda hacer una exploración de datos para encontrar estos errores. Os confirmo que hay unos cuantos!

Tanto "price" como "sells" intenté reasignarlos con df[columna].astype(type) para evitar el warning, pero en ambos casos no funcionó y se mantenía en tipo object, por lo que hice la asignación normal con df[columna] = df[columna].astype(type)

#### Separar múltiples desarrolladoras en varias listas

In [16]:
df.loc[:, 'developer'] = df['developer'].str.split(";")

In [17]:
df = df.explode('developer').reset_index(drop=True)

#### Eliminar carácteres especiales de "developer"

In [18]:
pat = r'[^\w\s]|_'

In [19]:
df.loc[:, 'developer'] = df['developer'].str.replace(pat, ' ', regex=True)

In [20]:
df.loc[:, 'developer'] = df['developer'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)

In [21]:
df.loc[:, 'developer'] = df['developer'].str.replace(r'\s+', ' ', regex=True)

In [22]:
df.loc[22980, 'developer'] = df.loc[22981, 'developer']

In [23]:
df = df.drop_duplicates()

#### Eliminar espacios extras en "name" y "developer"

In [47]:
name_espacios = df['name'] != df['name'].str.strip()

In [48]:
name_espacios.unique()

array([False])

In [26]:
df.loc[:, 'name'] = df['name'].str.strip()

In [49]:
developer_espacios = df['developer'] != df['developer'].str.strip()

In [50]:
developer_espacios.unique()

array([False,  True])

In [29]:
df.loc[:, 'developer'] = df['developer'].str.strip()

In [51]:
df[df['developer'].str.strip() != df['developer']]

,name,release_date,developer,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price €,sells,action,adventure,rpg,simulation,strategy,racing,early_access
25003,The Battle Of Ages,2018-09-07,NaN,0,2,1,0,3.99,10000,True,False,True,False,False,False,True


#### Normalizar nombres en "developer"

##### Intento Fallido

Intenté crear un diccionario de equivalencias que afecte a todas las desarrolladoras en el DataFrame, le di muchas vueltas al problema para poder solucionarlo, pero no pude. Son demasiadas desarrolladoras distintas, muchas con nombres muy parecidos entre si, pero que no son una misma desarrolladora, o con palabras en común que dificultaba la diferenciación entre las que pertenecen y la que no a una misma desarrolladora. Dejo igualmente el código que estuve probando, del cual agradecería tener una correción y alguna solución a este problema, ya que no quedé contento teniendo que resolver este problema a medias, de ser posible me gustaría poder arreglarlo aunque sea posterior a la correción incluso. Muchas gracias!

In [31]:
"""devs = set(df['developer'].unique())
devs_lower = {dev.lower(): dev for dev in devs if isinstance(dev, str)}
matches = {}
processed = set()"""

"devs = set(df['developer'].unique())\ndevs_lower = {dev.lower(): dev for dev in devs if isinstance(dev, str)}\nmatches = {}\nprocessed = set()"

In [32]:
"""for dev in devs:
    if dev in processed or pd.isna(dev):
        continue
    close_matches = process.extract(dev.lower(), devs_lower.keys(), limit=10, scorer=fuzz.ratio)
    group = [name for name, score in close_matches if score > 85]
    
    if len(set(group)) > 1:
        reference = min(group, key=len)
        for name in group:
            matches[devs_lower[name]] = devs_lower[reference]
        processed.update(group)"""

'for dev in devs:\n    if dev in processed or pd.isna(dev):\n        continue\n    close_matches = process.extract(dev.lower(), devs_lower.keys(), limit=10, scorer=fuzz.ratio)\n    group = [name for name, score in close_matches if score > 85]\n    \n    if len(set(group)) > 1:\n        reference = min(group, key=len)\n        for name in group:\n            matches[devs_lower[name]] = devs_lower[reference]\n        processed.update(group)'

##### Desarrolladoras más importante

Cree este diccionario de forma manual para solucionar al menos una parte del problema y poder continuar con el trabajo. Se que no es lo óptimo, pero es lo más que pude hacer, realmente esta parte de normalización de las desarrolladoras me excedió.

In [33]:
developer_mapping = {
    'CAPCOM Co Ltd': 'Capcom', 
    'Capcom': 'Capcom', 
    'CAPCOM CO LTD': 'Capcom', 
    'Capcom Vancouver': 'Capcom', 
    'Capcom U S A Inc': 'Capcom', 
    'Capcom Game Studio Vancouver': 'Capcom', 
    'CAPCOM': 'Capcom', 
    'Capcom Game Studio Vancouver Inc': 'Capcom',
    'Ubisoft': 'Ubisoft', 
    'Ubisoft Montreal': 'Ubisoft', 
    'Ubisoft Bulgaria': 'Ubisoft', 
    'Ubisoft Montpellier': 'Ubisoft', 
    'Ubisoft Romania': 'Ubisoft', 
    'Ubisoft Reflections': 'Ubisoft', 
    'Ubisoft Shanghai': 'Ubisoft', 
    'Ubisoft San Francisco': 'Ubisoft', 
    'Ubisoft Paris': 'Ubisoft', 
    'Ubisoft Montreal Massive Entertainment and Ubisoft Shanghai': 'Ubisoft', 
    'Ubisoft Toronto': 'Ubisoft', 
    'Ivory Tower in collaboration with Ubisoft Reflections': 'Ubisoft', 
    'RedLynx in collaboration with Ubisoft Shanghai Ubisoft Kiev': 'Ubisoft', 
    'Ubisoft Sofia': 'Ubisoft', 
    'Ubisoft Montreal Red Storm Shanghai Toronto Kiev': 'Ubisoft', 
    'Ubisoft Quebec': 'Ubisoft', 
    'Ubisoft Kiev': 'Ubisoft', 
    'Reflections a Ubisoft Studio': 'Ubisoft', 
    'Ubisoft Quebec in collaboration with Ubisoft Annecy Bucharest Kiev Montreal Montpellier Shanghai Singapore Sofia Toronto studios': 'Ubisoft', 
    'Ubisoft Montreal Studio': 'Ubisoft', 
    'Ubisoft Pune': 'Ubisoft', 
    'Ubisoft Annecy': 'Ubisoft', 
    'Ubisoft Bucharest': 'Ubisoft', 
    'Ubisoft Milan': 'Ubisoft', 
    'Ubisoft Belgrade': 'Ubisoft', 
    'Ubisoft Entertainment': 'Ubisoft', 
    'Ubisoft Singapore': 'Ubisoft', 
    'Ubisoft Blue Byte': 'Ubisoft',
    'Valve': 'Valve', 
    'Valve Corporation Nexon Korea Corporation': 'Valve',
    'BANDAI NAMCO Studio Inc': 'Bandai Namco', 
    'NAMCO BANDAI Games': 'Bandai Namco', 
    'BANDAI NAMCO Entertainment Inc': 'Bandai Namco', 
    'BANDAI NAMCO Studios Inc': 'Bandai Namco', 
    'BANDAI NAMCO Studio': 'Bandai Namco', 
    'BANDAI NAMCO Studios Vancouver': 'Bandai Namco', 
    'BANDAI NAMCO Studios': 'Bandai Namco',
    'Square Enix': 'Square Enix', 
    'SQUARE ENIX': 'Square Enix', 
    'Square Enix Montreal': 'Square Enix',
    'SEGA': 'SEGA', 
    'SEGA Studios Australia': 'SEGA', 
    'SEGA Hardlight': 'SEGA',
    '2K Czech': '2K', 
    '2K Marin': '2K', 
    '2K Australia': '2K', 
    '2K Boston': '2K', 
    '2K China': '2K',
    'FromSoftware': 'FromSoftware', 
    'FromSoftware Inc': 'FromSoftware',
    'BioWare Corporation': 'BioWare', 
    'BioWare': 'BioWare'
}

In [34]:
df['developer'] = df['developer'].replace(developer_mapping)

In [35]:
df = df.drop_duplicates()

In [36]:
len(df)

29285

#### Convertir "release_date" a datetime

In [37]:
df['release_date'] = pd.to_datetime(df['release_date'])

#### Quitar € de "price" y pasarlo a float

In [38]:
df.rename(columns={'price': 'price €'}, inplace=True)

In [39]:
df['price €'] = df['price €'].str.replace("€", "").astype(float)

#### Convertir "sells" a int

In [40]:
df['sells'] = df['sells'].astype('int64')

## Filtrar outliers 

## Lidiar con NANs

## Validar nuestros datos

# EDA

## Cual es el TOP10 de juegos más jugados?

## Cuál es el género de videojuegos más vendido?

## Qué géneros tienen mejores reviews de media?

## Cuánto ha ganado el desarrollador que más juegos ha vendido?

## Cómo afecta a las reviews que un juego sea Early Access?

## Cuál es el desarrollador (o desarrolladores) que más tipos de juego y cantidad ha desarrollado?

## En qué mes, desde que tenemos datos, se han publicado más juegos?

## Cuántas horas se juega de media a cada tipo de juego mensualmente?

## Los juegos Free to Play tienen mejores o peores reviews?

## Cómo ha evolucionado el número de achievements (trofeos) por juego?

## Bonus! Si tienes curiosidad, y quieres contestar más preguntas, puedes resolverlas aquí mismo!